In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/babble/data/')
# os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_spouse'
os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_bike'


from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

# Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
# candidate_class = Spouse

Biker = candidate_subclass('Biker', ['person', 'bike'])
candidate_class = Biker

In [3]:
candidates = session.query(candidate_class).filter(candidate_class.split == 0).all()
print(len(candidates))

2406


In [4]:
from pprint import pprint

In [5]:
import os
from mturk_processing import MTurkHelper
helper = MTurkHelper(candidates, num_hits=40, workers_per_hit=3)

In [6]:
output_csv_path= os.environ['SNORKELHOME'] + '/tutorials/babble/bike/data/VisualGenome_all_out.csv'
explanations = helper.postprocess_visual(output_csv_path, verbose=False)

Num HITs unique: 40
Num HITs total: 120
Unanimous: 99
Majority: 33
Bad: 27


In [7]:
# exp_iterator = iter(explanations)

In [8]:
# exp = exp_iterator.next()
# from snorkel.viewer import SentenceNgramViewer
# print(exp.condition)
# sv = SentenceNgramViewer([exp.candidate], session, n_per_page=3, height=150)
# sv

In [9]:
for exp in explanations[:5]: print(exp.condition)

Box X barely overlaps Box Y.
Box X and Box Y barely overlap.
Box X is always a person and outlined in blue and Box Y is always a bicycle outlined in yellow.
You can see the man standing up and peddling the bike.
the boxes are perfect


In [10]:
# from snorkel.contrib.babble import Explanation
# exp = Explanation("person1 equals person2", True, name='test_exp')
# homemades = [exp]

In [11]:
from snorkel.contrib.babble import Babbler
user_lists = {}
babbler = Babbler(mode='image', 
                  candidate_class=candidate_class, 
                  explanations=explanations)
# babbler.apply(parallelism=1)

Created grammar with 441 rules


In [12]:
lfs = babbler.generate_lfs()

358 parses created from 244 out of 367 explanation(s)


In [13]:
babbler.filter_duplicate_semantics()

Filtered to 92 LFs with duplicate semantics filter (266 filtered).


In [14]:
babbler.filter_consistency()

Filtered to 58 LFs with consistency filter (34 filtered).


In [15]:
%time babbler.generate_label_matrix(split=1, parallelism=1)

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 14.4 s, sys: 1.09 s, total: 15.5 s
Wall time: 25.4 s


<1037x58 sparse matrix of type '<type 'numpy.int64'>'
	with 37896 stored elements in Compressed Sparse Row format>

In [16]:
# babbler.load_matrix(session, split=1)

In [17]:
babbler.filter_uniform_signatures()

Filtered to 47 LFs with uniform signatures filter (11 filtered).


In [18]:
babbler.filter_duplicate_signatures()

Filtered to 30 LFs with duplicate signatures filter (17 filtered).


In [19]:
babbler.get_parses(translate=True)

['return -1 if call(smaller(Box X), Box Y) else 0',
 'return -1 if call(left(Box X), Box Y) else 0',
 "return -1 if call(below(edge(Box X, 'bottom')), edge(Box Y, 'bottom')) else 0",
 "return 1 if call(near(center(Box X)), edge(Box Y, 'bottom')) else 0",
 'return 1 if call(taller(Box X), Box Y) else 0',
 'return 1 if call(wider(Box Y), Box X) else 0',
 "return -1 if call(left(center(Box X)), edge(Box Y, 'right')) else 0",
 "return 1 if call(near(edge(Box Y, 'bottom')), edge(Box X, 'bottom')) else 0",
 "return -1 if call(above(edge(Box Y, 'bottom')), edge(Box X, 'bottom')) else 0",
 "return 1 if call(near(edge(Box Y, 'left')), edge(Box X, 'left')) else 0",
 "return 1 if call(near(edge(Box Y, 'right')), edge(Box X, 'right')) else 0",
 "return 1 if call(left(edge(Box Y, 'right')), edge(Box X, 'right')) else 0",
 "return -1 if call(right(edge(Box X, 'left')), edge(Box Y, 'right')) else 0",
 "return 1 if call(above(edge(Box X, 'bottom')), edge(Box Y, 'bottom')) else 0",
 'return -1 if call(

In [20]:
babbler.get_explanations()

[Explanation("Explanation11: False, Box X is a lot smaller than Box Y."),
 Explanation("Explanation12: False, Box X is to the left of Box Y."),
 Explanation("Explanation16: False, The bottom edge of Box X is below the bottom edge of Box Y."),
 Explanation("Explanation177: True, The bottom edge of box X is at the center of box Y."),
 Explanation("Explanation18: True, Box X", "Box X is taller than Box Y"),
 Explanation("Explanation19: True, Box Y is wider than box X"),
 Explanation("Explanation202: False, The right edge of Box X is left of the center of Box Y."),
 Explanation("Explanation25: True, The bottom of box Y is at the same place as the bottom of box X."),
 Explanation("Explanation257: False, The bottom edge of box Y is just above the bottom edge of box X."),
 Explanation("Explanation262: True, The left and right edges of box Y are very close to the left and right edges of box X."),
 Explanation("Explanation262: True, The left and right edges of box Y are very close to the left a

In [21]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

<1037x1 sparse matrix of type '<type 'numpy.int64'>'
	with 906 stored elements in Compressed Sparse Row format>

In [22]:
babbler.label_matrix.lf_stats(session, labels=L_gold_dev)

,j,Coverage,Overlaps,Conflicts,TP,FP,FN,TN,Empirical Acc.
Explanation5_0,0,0.274831,0.274831,0.274831,154,87,0,0,0.639004
Explanation6_0,1,0.400193,0.400193,0.400193,0,0,84,235,0.736677
Explanation11_0,2,0.369335,0.369335,0.369335,0,0,68,268,0.797619
Explanation12_0,3,0.493732,0.493732,0.493732,0,0,98,341,0.776765
Explanation16_0,4,0.377049,0.377049,0.377049,0,0,15,339,0.957627
Explanation18_0,5,0.814851,0.814851,0.814851,202,541,0,0,0.271871
Explanation19_0,6,0.505304,0.505304,0.505304,109,354,0,0,0.235421
Explanation25_0,7,0.620058,0.620058,0.620058,184,351,0,0,0.343925
Explanation29_1,8,0.994214,0.994214,0.994214,0,0,210,690,0.766667
Explanation31_0,9,0.494696,0.494696,0.494696,0,0,103,350,0.772627
